In [1]:
import json 
import threading
import time

from joblib import load
import pandas as pd
from datetime import datetime
from itertools import combinations

from flask import Flask, render_template, jsonify
from flask_cors import CORS

from api_scripts.binance import get_binance_prices 
from api_scripts.coin_gecko import get_coingecko_prices 
from api_scripts.coin_layer import get_coinlayer_prices, get_coinlayer_prices_yesterday 

In [27]:

def opportunity(seller_market,buy_price,buyer_market,sell_price,coin):
    return{"coin":coin,
    "sellerMarket":seller_market,
    "buyPrice":buy_price,
    "buyerMarket":buyer_market,
    "sellPrice":sell_price }
    
def compare(m1,m2,coin): #add number formatting and threshold
    m2price = coin_data[m2][coin]
    m1price = coin_data[m1][coin]

    if(m1price == m2price):
        return False
    
    if(m1price > m2price): 
        return opportunity(m1,m1price,m2,m2price,coin)
    else:
        return opportunity(m2,m2price,m1,m1price,coin)


def generate_arbitrage_opportunities(stale):
    if stale == False:
        cgd = get_coingecko_prices()
        bd = get_binance_prices()
        clfd = get_coinlayer_prices()["rates"]
    else:
        with open("jsons/cgarb.json","r") as f:
            cgd = json.load(f)
        with open("jsons/barb.json","r") as f:
            bd = json.load(f)
        with open("jsons/clarb.json","r") as f:
            clfd = json.load(f)["rates"]

    clf = {"Name": "CoinLayer","data": clfd}
    cgf = {"Name": "CoinGecko","data": format_cgd(cgd)}
    bf = {"Name": "Binance","data": format_bd(bd)}

    opportunities = []
    for coin in clf["data"]:

        comps = [compare(clf,cgf,coin),
                 compare(clf,bf,coin),
                 compare(cgf,bf,coin)]
            
        for comp in comps:
            if comp != False:
                opportunities.append(comp)
    return opportunities


def get_coin_prices(stale, coins,markets):
    prices_by_market = {}
    
    if stale == False:
        for market in markets:
            prices_by_market[market] = markets[market](coins) 
    else:
        for market in markets:
            with open(f"jsons/{market}_arb.json","r") as f:
                prices_by_market[market] = json.load(f)

    return prices_by_market
    

def gao2():
    opportunities = []
    for coin in coins:
        comps = []
        for market_pair in combinations(markets, 2):
            comps.append(compare(market_pair[0],market_pair[1],coin))
        for comp in comps:
            if comp != False:
                opportunities.append(comp)
    return opportunities

def get_yesterdays_data(stale):
    if stale == True:
        with open("jsons/clarbm2.json","r") as cl:
            yesterdata = json.load(cl)
    else:
        yesterdata = get_coinlayer_prices_yesterday()
        
    
    return formatted_data

def retrieveModels():
    coins = ['BTC','LTC','ETH','ADA','USDT']
    mls = {}
    for coin in coins:
        mls[coin] = load(f"ml/regressor_{coin}.joblib") 
    return mls

def model_preds():
    final_pred = {}
    coin_data = data2["yester"]
    for coin in coin_data:
        final_pred[coin] = list(models[coin].predict(pd.DataFrame(coin_data[coin])))[0]
    return final_pred

TypeError: make() takes 0 positional arguments but 1 was given

In [28]:
models = retrieveModels()   
coins = ["BTC","LTC","ETH","ADA","USDT"]
markets = {
        "CoinGecko" : get_coingecko_prices,
        "CoinLayer" : get_coinlayer_prices,
        "Binance" : get_binance_prices
    }
coin_data = get_coin_prices(True,coins,markets)

data2 = {"data":gao2(),"pred_data":format_yesterdays_data(True)}
print(gao2())


[{'coin': 'BTC', 'sellerMarket': 'CoinLayer', 'buyPrice': 27219.63083, 'buyerMarket': 'CoinGecko', 'sellPrice': 27181}, {'coin': 'BTC', 'sellerMarket': 'Binance', 'buyPrice': 27202.9, 'buyerMarket': 'CoinGecko', 'sellPrice': 27181}, {'coin': 'BTC', 'sellerMarket': 'CoinLayer', 'buyPrice': 27219.63083, 'buyerMarket': 'Binance', 'sellPrice': 27202.9}, {'coin': 'LTC', 'sellerMarket': 'CoinLayer', 'buyPrice': 95.992909, 'buyerMarket': 'CoinGecko', 'sellPrice': 95.53}, {'coin': 'LTC', 'sellerMarket': 'Binance', 'buyPrice': 95.98, 'buyerMarket': 'CoinGecko', 'sellPrice': 95.53}, {'coin': 'LTC', 'sellerMarket': 'CoinLayer', 'buyPrice': 95.992909, 'buyerMarket': 'Binance', 'sellPrice': 95.98}, {'coin': 'ETH', 'sellerMarket': 'CoinLayer', 'buyPrice': 1900.257044, 'buyerMarket': 'CoinGecko', 'sellPrice': 1898.42}, {'coin': 'ETH', 'sellerMarket': 'Binance', 'buyPrice': 1900.42, 'buyerMarket': 'CoinGecko', 'sellPrice': 1898.42}, {'coin': 'ETH', 'sellerMarket': 'Binance', 'buyPrice': 1900.42, 'buye

In [9]:
app = Flask(__name__)

CORS(app,resources={r"/*": {"origins": ["https://turkishfilms.github.io"]}})

def get_data_on_delay():
    while True:
        data2["data"] = generate_arbitrage_opportunities(False)
        data2["ops"] = gao2(True)
        data2["yester"] = get_yesterdays_data(False)
        data2["pred_data"] = model_preds()
        time.sleep(14400) 

@app.route('/')
def home():
   return render_template('index.html',data=data2["ops"])

@app.route('/learn')
def a():
    return render_template('learn.html')

@app.route('/predictions')
def b():
    return render_template('predictions.html',data=json.dumps(data2["pred_data"]))

if(__name__ == "__main__"):
    t = threading.Thread(target=get_data_on_delay)
    t.start()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jun/2023 08:22:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 08:22:22] "GET /favicon.ico HTTP/1.1" 404 -
